STEAM_GAMES

Se importan las librerias a usar

In [184]:
# se importan las librerias que se van a usar 
import gzip
import ast
import chardet
import pandas as pd
import nltk

Se abre el archivo json comprimido que contiene la informacion de steam_game

In [185]:
# se define la ruta donde esta ubicado el archivo
file_path = '../datasets/steam_games.json.gz'
# se lee el archivo json
steam_games = pd.read_json(file_path, compression='gzip', lines=True)
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [186]:
steam_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se identifica las columnas que se van a usar para la creacion de las funciones solicitadas  para los endpoints, por lo cual se procede a hacer la limpieza de la Data. primero se eliminaran las filas que solo contienen valores nulos.

In [187]:
steam_games.dropna(how="all", inplace=True  )
steam_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

Para cambiar el tipo de dato en la columna id se identifico que contiene valores nulos, se procede a verificar.

In [188]:
steam_games[steam_games["id"].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


la fila 88384 solo contiene la url, parece no ser relevante para el analisis por ello se elimina, para la fila 119271 se filtrara para identificar si tiene un duplicado ya que contiene casi toda la informacion y nos puede servir para el analisis posterior

In [189]:
title = "Batman: Arkham City - Game of the Year Edition"
developer = "Rocksteady Studios,Feral Interactive (Mac)"
filtro = steam_games[(steam_games["title"]==title) & (steam_games["developer"]==developer)]
filtro

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


se encontro otra fila con la misma informacion pero con su id, por ello se procede a eliminar la fila que no posee id

In [190]:
steam_games.dropna(subset=['id'], inplace=True) #elimina filas de la columna id que son null

se verifica que la columna id no tenga duplicados

In [191]:
steam_games[steam_games.duplicated(subset=['id'], keep=False)]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


ya que la columna id no deberia contener duplicados se procede a eliminar una de las filas

In [192]:
steam_games.drop_duplicates(subset="id", inplace=True, keep="last")

con los cambios aplicados ya se puede cambiar el tipo de dato

In [193]:
steam_games["id"] = steam_games["id"].astype(int)

verificamos los nulos en el df

In [194]:
steam_games.isnull().sum()

publisher       8051
genres          3282
app_name           1
title           2049
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
id                 0
developer       3298
dtype: int64

Verificamos los nulos en la columana de genres que se usara mas adelante en las fuinciones, ya que la columna tags contiene casi la misma informacion verificamos los nulos que quedaran si cruzamos la informacion 

In [195]:
steam_games[(steam_games["genres"].isnull()) & (steam_games["tags"].isnull())].shape

(138, 13)

Se reemplaza los nulos por los datos de la columna tags ya que en algunas filas el contenido es igual y solo cambia el orden y en otras muy similar.

In [196]:
# Crea un conjunto de géneros únicos 
unique_genres = set(item for val in steam_games['genres'].dropna() for item in val)

# Filtra las etiquetas 'tags' 
steam_games['tags'] = steam_games['tags'].apply(lambda x: [item for item in x if item in unique_genres] if isinstance(x, list) else x) 

# Rellena valores nulos en la columna 'genres' 
steam_games['genres'].fillna(steam_games['tags'], inplace=True)

def add_tags_to_genres(row): 
    ''' Esta función añade valores de 'tags' a 'genres' que no estén ya'''
    genres = row['genres']
    tags = row['tags']
    if isinstance(tags, list) and isinstance(genres, list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres

#aplico la funcion
steam_games['genres'] = steam_games.apply(lambda row: add_tags_to_genres(row), axis=1)

# elimina columna tags
steam_games.drop(columns=['tags'], inplace=True)

eliminamos los nulos de la columna genres ya que no se puede tratar

In [197]:
(steam_games['genres'].isnull().sum() / steam_games.shape[0])*100 #porcentaje de nulos

0.42947840159342715

In [198]:
steam_games['genres'].isnull().sum()

138

In [199]:
steam_games = steam_games[steam_games['genres'].notnull()]# se eliminan los nulos que quedaron en la columna genres

revisamos la columna release_date que es necesaria para hacer la primer funcion

In [200]:
# Contar la frecuencia de cada valor único en "release_date"
steam_games["release_date"].value_counts()

release_date
2012-10-16            100
2017-08-31             92
2017-09-26             89
2017-06-21             82
2017-07-25             78
                     ... 
2017 Q4                 1
Play Beta in demo!      1
1970-07-15              1
1975-12-31              1
2018-10-01              1
Name: count, Length: 3579, dtype: int64

Extraemos el año que vamos a usar en la funcion, para ello creamos la columna year

In [201]:
# Función para extraer el año
def extraer_año(fecha):
    try:
        # Intenta extraer un año de la cadena
        año = pd.to_datetime(fecha, errors='coerce').year
        return int(año) if not pd.isnull(año) else None
    except:
        return None

# Aplicar la función para crear la nueva columna 'año'
steam_games['year'] = steam_games['release_date'].apply(extraer_año)



C:\Users\Willc\AppData\Local\Temp\ipykernel_16184\1571513962.py:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  año = pd.to_datetime(fecha, errors='coerce').year


verificamos la proporcion de nulos, ya que no hay tratamiento para estos

In [202]:
steam_games['year'].isnull().sum()

2239

In [203]:
(steam_games['year'].isnull().sum() / steam_games.shape[0])*100 #porcentaje de nulos

6.998187160092517

Se eliminan estos datos nulos ya que no representa mucho del total del data set, pero antes se crea un df con la informacion de los id y title que no tengan nulos, ya que la idea es no perder informacion importante que nos servira para el analisis 

In [206]:
steam_games_id_title = steam_games[['id', 'title']]#se crea el df solo con id y title

In [207]:
steam_games_id_title = steam_games_id_title[steam_games_id_title['title'].notnull()]#se eliminan las filas nulas de la columna title

In [167]:
steam_games = steam_games[steam_games['year'].notnull()]#eliminamos las filas nulas de la columna year

Se verifica la fila null del la columna title

In [168]:
steam_games[steam_games['title'].isnull()]

,publisher,genres,app_name,title,url,release_date,reviews_url,specs,price,early_access,id,developer,year
90890,None,"[Action, Indie]",None,None,http://store.steampowered.com/app/317160/_/,2014-08-26,http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",None,0.0,317160,None,2014.0


Se elimina la fila ya que no aporta al analisis 

In [169]:
steam_games = steam_games[steam_games['title'].notnull()]

La columna Publisher y la columna developer contienen casi la misma informacion, por ello se precede traer los valores que no estan en publisher pero si en developer

In [170]:
steam_games['publisher'].fillna(steam_games['developer'], inplace=True)#Remplaza los valores nulos de la columna publisher con los valores de la columna developer

Ahora verificamos los datos de la columna price

In [171]:
steam_games['price'].value_counts()

price
4.99           4041
9.99           3615
2.99           3321
0.99           2499
1.99           2425
               ... 
995.0             1
71.7              1
38.85             1
320.0             1
Free to Use       1
Name: count, Length: 156, dtype: int64

In [172]:
non_float_price_rows = steam_games[steam_games['price'].apply(lambda x: not isinstance(x, float))]

In [173]:
non_float_price_rows['price'].value_counts()

price
Free                             710
Free to Play                     472
Free To Play                     370
Free Mod                           4
Free Demo                          3
Play Now                           2
Third-party                        2
Play for Free!                     2
Play WARMACHINE: Tactics Demo      1
Install Theme                      1
Install Now                        1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Starting at $499.00                1
Starting at $449.00                1
Free to Try                        1
Free Movie                         1
Free to Use                        1
Name: count, dtype: int64

In [174]:
steam_games['price'] = steam_games['price'].replace(['f', 'F', 'I', 'T', 'P'], 0, regex=True)#Reemplazamos los valores que contengan string por 0

Para las filas que contienen Starting at $499.00  y Starting at $449.00 realizamos la siguiente funcion para que el valor de la columna price sea la parte float

In [175]:
# Definir una función para extraer la parte numérica de una cadena y sobrescribir la columna "price"
def extract_and_replace_numeric_price(string):
    if isinstance(string, str) and string.startswith('Starting at $'):
        numeric_part = string[len('Starting at $'):]
        try:
            numeric_value = float(numeric_part)
            return numeric_value
        except ValueError:
            return string  # Si no se puede convertir a float, devolver la cadena original
    else:
        return string  # Si no es una cadena que comienza con "Starting at $", no hacer cambios

In [176]:
# Aplicar la función a la columna "price" para sobrescribir los valores con la parte numérica extraída
steam_games['price'] = steam_games['price'].apply(extract_and_replace_numeric_price)

In [177]:
steam_games['price'] = steam_games['price'].astype(float)#Cambiamos a tipo de dato Float

Se eliminan las columnas que no se usaran en el analisis

In [178]:
steam_games.drop(columns=['developer','url','reviews_url','early_access','specs'], inplace=True)

Se verifica que no hay valores nulos en el df

In [179]:
steam_games.isnull().sum()

publisher       1161
genres             0
app_name           0
title              0
release_date       0
price           1054
id                 0
year               0
dtype: int64

Se verifica la proporcion de nulos antes de eliminarlos, ya que no se pueden aplicar mas cambios

In [180]:
(steam_games['publisher'].isnull().sum()/steam_games.shape[0])*100

3.901996370235935

In [181]:
steam_games = steam_games.dropna()# se eliminan los valores nulos

se guarda como csv 

In [182]:
steam_games.to_csv('../datasets/steam_games.csv',index=False)

Se crear un csv con con todos los id y title que usaremos en nuestro modelo de recomendaciones, la idea es no eliminar ningun titulo

In [210]:
steam_games_id_title.to_csv('../datasets/steam_games_id_title.csv.gz',index=False)